In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install glob2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from glob import glob
import os
import pandas as pd
path = "/content/drive/MyDrive/Iqram Sir/Fitabase data pulls - seperate"

In [25]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from tqdm.notebook import tqdm
from datetime import datetime

#------------------------------------------------------------------------------------------------

def get_content(patient_folder_adrs):

  post_adrs = sorted(glob(patient_folder_adrs+"/*"))[0]
  

  pre_adrs = sorted(glob(patient_folder_adrs+"/*"))[1]

  for i in glob(post_adrs+"/*"):
    content = i.split("_")
    if "heartrate" in content and "seconds" in content:
      post_sec = i
    if "minuteMETsNarrow" in content:
      post_met = i


  for i in glob(pre_adrs+"/*"):
    content = i.split("_")
    if "heartrate" in content and "seconds" in content:
      pre_sec = i
    if "minuteMETsNarrow" in content:
      pre_met = i
 
  return post_sec,post_met, pre_sec, pre_met



def remove_sec(l):
  k = datetime.strptime(l, "%m/%d/%Y %I:%M:%S %p")
  out_time = datetime.strftime(k, "%-m/%-d/%Y %-I:%M %p")
  return out_time




def add_sec(l):
  k = datetime.strptime(l, "%m/%d/%Y %I:%M %p")
  out_time = datetime.strftime(k, "%-m/%-d/%Y %-I:%M:%S %p")
  return out_time



def sec_met(df_sec,df_met):
  li_met = list(df_met['ActivityMinute'])
  li_met_s = []
  li_sec = list(df_sec['Time'])

  for i in li_met:
    li_met_s.append(remove_sec(i))



  activity_time=[]
  hr=[]
  mets=[]

  print("Finalizing The DataFrame")
  for i in tqdm(li_sec):
    if remove_sec(i) in li_met_s:
      activity_time.append(i)
      hr.append(list(df_sec.query("Time == '"+i+"'")['Value'])[0])
      mets.append(list(df_met.query("ActivityMinute == '"+add_sec(remove_sec(i))+"'")['METs'])[0])
  

  d={
    "ActivityMinute" : activity_time,
    "HR(BPM)" : hr,
    "METs" : mets
  }

  dataFrame = pd.DataFrame.from_dict(d)
  return dataFrame


def find_freq(lis):
  frequency = {}
  for item in tqdm(lis):
    if item in frequency:
        frequency[item] += 1
    else:
        frequency[item] = 1
  return frequency





def hr_range_permin(dfSec,freq):
  lis = []

  print("Removing 'Second' Section....")
  for i in tqdm(list(dfSec["Time"])):
    lis.append(remove_sec(i))

  print("Finding Frequency of Each 'Minute'....")
  dic = find_freq(lis)

  print("Creating Removable List of 'Minutes'...")
  removable_list = []
  for i in tqdm(dic):
    if dic[i] < freq:
      removable_list.append(i)


  print("Configuring HR/Min...")
  for i in tqdm(list(dfSec["Time"])):

    if remove_sec(i) in removable_list:
      dfSec.drop(dfSec[dfSec['Time'] == i].index, inplace = True)

  dfSec = dfSec.reset_index(drop=True)
  print("Process Complete..")
  return dfSec





def mets_activity(dfMets,up,low):

  dfMets.drop(dfMets[dfMets['METs'] >up].index, inplace = True)
  dfMets.drop(dfMets[dfMets['METs'] <=low].index, inplace = True)
  dfMets = dfMets.reset_index(drop = True)
  return dfMets


def formate24(t):
  in_time = datetime.strptime(t, "%m/%d/%Y %I:%M:%S %p")
  out_time = datetime.strftime(in_time, "%m/%d/%Y %H:%M")
  return out_time


def dif_min(start_time, end_time):
  t1 = datetime.strptime(start_time, "%m/%d/%Y %H:%M")
  t2 = datetime.strptime(end_time, "%m/%d/%Y %H:%M")

  delta = t2 - t1

  return delta.total_seconds()/60



def range_list(k):
  list_of = []
  main_list = []

  for i in range(len(k)-1):
    if dif_min(formate24(k[i]),formate24(k[i+1])) == 0:
      continue
    if dif_min(formate24(k[i]),formate24(k[i+1])) == 1:
      list_of.append(k[i])
    else:
      list_of.append(k[i])
      main_list.append(list_of)
      list_of=[]
    if i == len(k)-2:
      if dif_min(formate24(k[i]),formate24(k[i+1])) == 1:
        list_of.append(k[i+1])
        main_list.append(list_of)
      else:
        main_list.append([k[i+1]])

  return main_list







def hr_engine(data,epoch,saving_path):

  li=[]

  for i in list(data["ActivityMinute"]):
    li.append(i)


  ranges = range_list(li)


  sub_li=[]

  for i in tqdm(ranges):
    if len(i)>epoch:
      st = 0
      end = epoch
      for j in range(int(len(i)/epoch)):
        sub_li.append(i[st:end])
        st=st+epoch
        end=end+epoch+1


  epoch_num = 1


  for j in tqdm(sub_li):
    hr=[]
    time = []
    for k in j:
      for i in range(len(li)):
        if remove_sec(k)==remove_sec(li[i]):
          hr.append(list(data.query("ActivityMinute == '"+li[i]+"'")['HR(BPM)'])[0])
          time.append(li[i])
    d={
      "ActivityMinute" : time,
      "HR(BPM)" : hr
    }
    df = pd.DataFrame(d)
    df.to_csv(saving_path+"_epoch_"+str(epoch_num)+".csv",index = False)
    epoch_num+=1


In [69]:
subject = '001-JV'



for i in glob(path+"/*"):
  if subject in i.split("/"):
    Post_sec,Post_met, Pre_sec,Pre_met = get_content(i)

df_post_sec = pd.read_csv(Post_sec)
df_post_met = pd.read_csv(Post_met)

df_pre_sec = pd.read_csv(Pre_sec)
df_pre_met = pd.read_csv(Pre_met)


In [20]:
data_saving_folder = "/content/drive/MyDrive/Iqram Sir/Final_Fitabase_pyHRV"

In [68]:
#@title Select METs Ranges { run: "auto" }
option = "10 \u003C METs \u003C= 20" #@param ["0 < METs <= 10","10 < METs <= 20" ,"20 < METs <= 40", "40 < METs <=60","60<METs"]
print('You selected', option)

if option == "0 \u003C METs \u003C= 10":
  upper_lim = 10
  lower_lim = 0
  activity = "Resting"
  

elif option == "10 \u003C METs \u003C= 20":
  upper_lim = 20
  lower_lim = 10
  activity = "Light_Activity"


elif option == "20 \u003C METs \u003C= 40":
  upper_lim = 40
  lower_lim = 20
  activity = "Moderate_Activity"

elif option == "40 \u003C METs \u003C=60":
  upper_lim = 60
  lower_lim = 40
  activity = "Heavy_Activity"

elif option == "60\u003CMETs":
  upper_lim = 10000
  lower_lim = 60
  activity = "Extreme_Activity"

print("Upper Limit: ",upper_lim)
print("Lower Limit: ",lower_lim)
print("Activity: ",activity)



You selected 10 < METs <= 20
Upper Limit:  20
Lower Limit:  10
Activity:  Light_Activity


In [70]:
%%time


hr_range_per_min = 5
condition = 'post'


df_post_sec = hr_range_permin(df_post_sec,hr_range_per_min)


df_post_met = mets_activity(df_post_met,upper_lim,lower_lim)

folder_name = "/content/drive/MyDrive/Iqram Sir/argent/"+subject+"_"+str(hr_range_per_min)+"_"+activity+"_"+condition+"_condition.csv"

d = sec_met(df_post_sec,df_post_met)
d.to_csv(folder_name,index =False)

Removing 'Second' Section....


  0%|          | 0/294630 [00:00<?, ?it/s]

Finding Frequency of Each 'Minute'....


  0%|          | 0/294630 [00:00<?, ?it/s]

Creating Removable List of 'Minutes'...


...

  0%|          | 0/294630 [00:00<?, ?it/s]

Process Complete..
Finalizing The DataFrame


  0%|          | 0/289134 [00:00<?, ?it/s]

  0%|          | 0/3536 [00:00<?, ?it/s]

  0%|          | 0/131 [00:00<?, ?it/s]

In [71]:
dat = pd.read_csv(folder_name)

In [72]:
number_of_epoch = 10

saving_folder = "/content/drive/MyDrive/Iqram Sir/argent/"+subject+"_"+str(hr_range_per_min)+"_"+activity+"_"+condition+"_condition"

hr_engine(dat,number_of_epoch,saving_folder)

  0%|          | 0/3536 [00:00<?, ?it/s]

  0%|          | 0/131 [00:00<?, ?it/s]

In [73]:
%%time


#hr_range_per_min = 4
condition = 'pre'


df_pre_sec = hr_range_permin(df_pre_sec,hr_range_per_min)


df_pre_met = mets_activity(df_pre_met,upper_lim,lower_lim)

folder_name = "/content/drive/MyDrive/Iqram Sir/argent/"+subject+"_"+str(hr_range_per_min)+"_"+activity+"_"+condition+"_condition.csv"

d = sec_met(df_pre_sec,df_pre_met)
d.to_csv(folder_name,index =False)

Removing 'Second' Section....


  0%|          | 0/350946 [00:00<?, ?it/s]

Finding Frequency of Each 'Minute'....


  0%|          | 0/350946 [00:00<?, ?it/s]

Creating Removable List of 'Minutes'...


...

Configuring HR/Min...


  0%|          | 0/350946 [00:00<?, ?it/s]

Process Complete..
Finalizing The DataFrame


  0%|          | 0/344139 [00:00<?, ?it/s]

CPU times: user 35min 8s, sys: 18.6 s, total: 35min 27s
Wall time: 35min 52s


In [74]:
dat = pd.read_csv(folder_name)

In [ ]:
#number_of_epoch = 10

saving_folder = "/content/drive/MyDrive/Iqram Sir/argent/"+subject+"_"+str(hr_range_per_min)+"_"+activity+"_"+condition+"_condition"

hr_engine(dat,number_of_epoch,saving_folder)

  0%|          | 0/4476 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]